In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
from nltk.tokenize import sent_tokenize
import csv
import string

In [3]:
def remove_scenes_actions(line):
    new = ""
    found = False
    for letter in line:
        if letter == "(" or letter =="[":
            found = True
        if letter == ")" or letter =="]":
            found = False

        #outside brackets
        if found == False and letter!=")" and letter!="]":
            new = new + letter
    
    return(new)

In [5]:

#docs = process_file_friends("1x01 - The One Where Monica Gets A Roommate.srt")
file = open("friends.csv", 'r', encoding="utf8")
csvreader = csv.reader(file)
data = list(csvreader)
characters =["Rachel", "Monica", "Joey", "Chandler", "Ross", "Phoebe"]
    
vectorizer = TfidfVectorizer()

In [17]:
values = []

for character in characters:
    lines = []
    sp_character = []
    for row in data:
        if row[-2] == character and row[0]=="S01" and row[1]=="E01":
            #we only want dialogue
            str = remove_scenes_actions(row[-1])
            lines.append(str)
    
    #create auxiliar vector WITHOUT punctuation for TF-IDF purposes
        
    for line in lines:
        #remve punctuation
        line = line.translate(str.maketrans('', '', string.punctuation))
        sp_character.append(line)

    #compute tf-idf
    tf_idf = vectorizer.fit_transform(sp_character)
    tf_idf_array = tf_idf.toarray()
    scores = []
        
    for sentence in tf_idf_array:
        value = 0
        for n in sentence:
            value+=n
        scores.append(value)

    final = []

    #format: final = [...,[sentence, tf-idf score],...]
    for i in range(len(scores)):
        cell = []
        #uses original array with punctuation
        cell= [scores[i], lines[i]]
        final.append(cell)
        
    sorted_list = sorted(final)
    l = []

    #get only the strings
    for elem in sorted_list:
        l.append(elem[1])

    #remove duplicates
    l = list( dict.fromkeys(l) )
    
    #counts goes from higher-> lower tf-idf
    counts = l[-15::]

    values.append(counts)
        
    '''print("----ten lowest scores----")
    for elem in l[:20]:
        print(elem)

    print("\n")
    print("----ten highest scores----")
    for elem in l[-10:]:
        print(elem)'''

dict = {key: value for key, value in zip(characters, values)}

In [18]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(dict)

# displaying the DataFrame
df.style

,Rachel,Monica,Joey,Chandler,Ross,Phoebe
0,"Daddy, I just... I can't marry him! I'm sorry. I just don't love him. Well, it matters to me!","My brother's going through that right now, he's such a mess. How did you get through it?","Phoebe, Ross, Chandler, and Push her down the stairs! Push her down the stairs! Push her down the stairs!","Okay, so, I'm in Las Vegas... I'm Liza Minelli-","No I don't, to hell with her, she left me!",What I said you had...
1,"Isn't this amazing? I mean, I have never made coffee before in my entire life.","So you wanna tell us now, or are we waiting for four wet bridesmaids?",I can't believe you didn't know it was a line!,"Then I look down, and I realize there's a phone... there.","I'll be fine, alright? Really, everyone. I hope she'll be very happy.","Oh, was I doing it again?"
2,"Look Daddy, it's my life. Well maybe I'll just stay here with Monica.",So how you doing today? Did you sleep okay? Talk to Barry? I can't stop smiling.,"Strip joint! C'mon, you're single! Have some hormones!","Please don't do that again, it's a horrible sound.",You did! Oh.... I always figured you just thought I was Monica's geeky older brother.,"Wait, does he eat chalk?"
3,Are you kidding? I'm trained for nothing! I was laughed out of twelve interviews today.,Maybe. Joey: Wait. Your 'not a real date' tonight is with Paul the Wine Guy?,That wasn't a real date?! What the hell do you do on a real date?,"Ooh, she should not be wearing those pants.","I told mom and dad last night, they seemed to take it pretty well.",I can't believe what I'm hearing here...
4,"Well, I was kinda supposed to be headed for Aruba on my honeymoon, so nothing!",I know being spit on is probably not what you need right now. Um... how long?,"You should both know, that he's a dead man. Oh, Chandler?","I'm sorry, I didn't catch your name. Paul, was it?","No!! Okay?! Why does everyone keep fixating on that? She didn't know, how should I know?","All right, c'mere, gimme your feet."
5,Y'know what? I think we can just leave it at that. It's kinda like a symbolic gesture...,"That's Paul's watch. You just put it back where you found it. Oh boy. Alright. Goodnight, everybody.","Hey-hey-hey-hey, if you're gonna start with that stuff we're outta here.",I would have to say that is an 'L'-shaped bracket.,I assume we're looking for an answer more sophisticated than 'to get you into bed'.,"Oh, I wish I could, but I don't want to."
6,"You would be too if you found John and David boots on sale, fifty percent off!","Oh wait, wait, unless you happened to catch the Reruns' production of Pinocchio, at the little theater in the park.","Great story! But, I uh, I gotta go, I got a date with Andrea--Angela--Andrea... Oh man,",Sometimes I wish I was a lesbian... Did I say that out loud?,"Okay. You know you probably didn't know this, but back in high school, I had a, um, major crush on you.","If I let go of my hair, my head will fall off."
7,"Oh God, come on you guys, is this really necessary? I mean, I can stop charging anytime I want.","De-caff. Okay, everybody, this is Rachel, another Lincoln High survivor. This is everybody, this is Chandler, and Phoebe, and Joey, and- you remember my brother Ross?","C'mon, you're going out with the guy! There's gotta be something wrong with him!","If can invade Poland, there isn't anything I can't do.","This was Carol's favorite beer. She always drank it out of the can, I should have known.","Ooh, I just pulled out four eyelashes. That can't be good."
8,"Well actually thanks, but I think I'm just gonna hang out here tonight. It's been kinda a long day.","Okay, look, this is probably for the best, y'know? Independence. Taking control of your life. The whole, 'hat' thing.","Oh, you wouldn't know a great butt if it came up and bit ya.","Y'know, if you listen closely, you can hear a thousand retailers scream.","Y'know, here's the thing. Even if I could get it together enough to- to ask a woman out,... who am I gonna ask?","What does that mean?